## Imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
projectdir = "/home/jovyan/work/MED_Fall"
workdir = "/home/jovyan/work"

In [3]:
import os

os.environ["PYTHONPATH"]

'/home/jovyan/work/MED_Fall/'

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from utils.utility_functions import listdir_nohidden_sorted as lsdir
from utils.utility_functions import load_images, show_images

In [5]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Setup the dataframe for keras flow_from_dataframe pipeline

In [34]:
extracted_frames_path = (
    "/home/jovyan/work/MED_Fall/vision/vision_dataset/extracted_frames/"
)

In [8]:
GROUND_TRUTH_PATH = "/home/jovyan/work/MED_Fall/vision/vision_dataset/ground_truth_new"

In [9]:
# Load all csv files (one per actor). Each file contains frames names and respective labels.
dfs = []
for file in lsdir(GROUND_TRUTH_PATH):
    df = pd.read_csv(file)
    dfs.append(df)
# concatenate all csv files in one pandas DF
dataset = pd.concat(dfs, ignore_index=True, axis=0)
dataset = dataset.iloc[:, 2:]
dataset

,micro_labels,macro_labels,ar_labels,frame_name
0,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0000
1,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0001
2,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0002
3,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0003
4,lie_still,lying_down,actor_repositioning,actor_1_bed_cam_1_0004
...,...,...,...,...
1182295,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4615
1182296,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4616
1182297,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4617
1182298,stand_up_from_floor,adl,actor_repositioning,actor_4_chair_full_ph_cam_7_4618


In [10]:
# Select only "on_air" frames (where actors are performing sequences)
dataset_onair = dataset.loc[dataset["ar_labels"] == "on_air"].copy()
dataset_onair

,micro_labels,macro_labels,ar_labels,frame_name
206,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0206
207,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0207
208,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0208
209,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0209
210,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0210
...,...,...,...,...
1182213,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4533
1182214,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4534
1182215,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4535
1182216,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4536


In [14]:
# check on_air dataset samples per class
dataset_onair["macro_labels"].value_counts()

adl           269836
falling        88312
lying_down     27958
Name: macro_labels, dtype: int64

In [15]:
# instantiate and fit a OneHotEncoder for the macro classes (adl, fall, lie_down)
le = LabelBinarizer()
le.fit(dataset_onair["macro_labels"])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping

{'adl': array([1, 0, 0]),
 'falling': array([0, 1, 0]),
 'lying_down': array([0, 0, 1])}

## Use actor 3 as val set

In [16]:
actor_3 = dataset_onair.loc[dataset_onair["frame_name"].str.contains("actor_3")].copy()
actor_4 = dataset_onair.loc[dataset_onair["frame_name"].str.contains("actor_4")].copy()

actor_34 = pd.concat([actor_3, actor_4], axis=0)
actor_34

,micro_labels,macro_labels,ar_labels,frame_name
823349,sit_up_from_lying,adl,on_air,actor_3_bed_cam_1_0149
823350,sit_up_from_lying,adl,on_air,actor_3_bed_cam_1_0150
823351,sit_up_from_lying,adl,on_air,actor_3_bed_cam_1_0151
823352,sit_up_from_lying,adl,on_air,actor_3_bed_cam_1_0152
823353,sit_up_from_lying,adl,on_air,actor_3_bed_cam_1_0153
...,...,...,...,...
1182213,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4533
1182214,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4534
1182215,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4535
1182216,crouched_still,falling,on_air,actor_4_chair_full_ph_cam_7_4536


In [17]:
actor_12 = dataset_onair.drop(actor_34.index)
actor_12

,micro_labels,macro_labels,ar_labels,frame_name
206,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0206
207,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0207
208,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0208
209,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0209
210,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0210
...,...,...,...,...
822862,fall_lateral,falling,on_air,actor_2_walk_stick_full_ph_cam_7_0982
822863,fall_lateral,falling,on_air,actor_2_walk_stick_full_ph_cam_7_0983
822864,fall_lateral,falling,on_air,actor_2_walk_stick_full_ph_cam_7_0984
822865,fall_lateral,falling,on_air,actor_2_walk_stick_full_ph_cam_7_0985


In [18]:
train_set = actor_12

In [19]:
train_set["macro_labels"].value_counts()

adl           182630
falling        60851
lying_down     23548
Name: macro_labels, dtype: int64

In [20]:
val_set = actor_3
val_set["macro_labels"].value_counts()

adl           61502
falling       14469
lying_down     3738
Name: macro_labels, dtype: int64

## Balance dataset

In [21]:
# dataset_balanced = dataset_onair.copy()  # make a copy of on_air dataset

counts = train_set["macro_labels"].value_counts()  # count samples per class
minority_class_samples = counts[-1]  # store minority class samples

# select adl samples
adl_samples = train_set.loc[dataset["macro_labels"] == "adl"]

# select fall samples
fall_samples = train_set.loc[dataset["macro_labels"] == "falling"]

## subsample

# count exceeding adl samples wrt minority_class_samples
adl_todrop_ind = len(adl_samples) - minority_class_samples
# count exceeding fall samples wrt minority_class_samples
fall_todrop_ind = len(fall_samples) - minority_class_samples

# select indices to drop from adl subset
adl_samples_to_drop = adl_samples.iloc[-adl_todrop_ind:]

# select indices to drop from fall subset
fall_samples_to_drop = fall_samples.iloc[-fall_todrop_ind:]

# actually drop exceeding adl samples
train_set.drop(adl_samples_to_drop.index, axis=0, inplace=True)

# actually drop exceeding fall samples
train_set.drop(fall_samples_to_drop.index, axis=0, inplace=True)

# count samples per class of the balanced dataset
train_set["macro_labels"].value_counts()

adl           23548
falling       23548
lying_down    23548
Name: macro_labels, dtype: int64

In [22]:
train_set.sort_index(inplace=True)

## OneHotEncode Labels

In [24]:
onehotlabels = list(le.transform(train_set["macro_labels"]))
train_set["onehotlabels"] = onehotlabels
train_set

,micro_labels,macro_labels,ar_labels,frame_name,onehotlabels
206,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0206,"[1, 0, 0]"
207,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0207,"[1, 0, 0]"
208,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0208,"[1, 0, 0]"
209,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0209,"[1, 0, 0]"
210,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0210,"[1, 0, 0]"
...,...,...,...,...,...
810180,lie_down_on_the_floor,lying_down,on_air,actor_2_walk_ph_cam_7_1260,"[0, 0, 1]"
810181,lie_down_on_the_floor,lying_down,on_air,actor_2_walk_ph_cam_7_1261,"[0, 0, 1]"
810182,lie_down_on_the_floor,lying_down,on_air,actor_2_walk_ph_cam_7_1262,"[0, 0, 1]"
810183,lie_down_on_the_floor,lying_down,on_air,actor_2_walk_ph_cam_7_1263,"[0, 0, 1]"


In [26]:
val_onehotlabels = list(le.transform(val_set["macro_labels"]))
val_set["onehotlabels"] = val_onehotlabels
val_set

,micro_labels,macro_labels,ar_labels,frame_name,onehotlabels
823349,sit_up_from_lying,adl,on_air,actor_3_bed_cam_1_0149,"[1, 0, 0]"
823350,sit_up_from_lying,adl,on_air,actor_3_bed_cam_1_0150,"[1, 0, 0]"
823351,sit_up_from_lying,adl,on_air,actor_3_bed_cam_1_0151,"[1, 0, 0]"
823352,sit_up_from_lying,adl,on_air,actor_3_bed_cam_1_0152,"[1, 0, 0]"
823353,sit_up_from_lying,adl,on_air,actor_3_bed_cam_1_0153,"[1, 0, 0]"
...,...,...,...,...,...
1013767,lie_down_on_the_floor,lying_down,on_air,actor_3_walk_stick_full_ph_cam_7_5767,"[0, 0, 1]"
1013768,lie_down_on_the_floor,lying_down,on_air,actor_3_walk_stick_full_ph_cam_7_5768,"[0, 0, 1]"
1013769,lie_down_on_the_floor,lying_down,on_air,actor_3_walk_stick_full_ph_cam_7_5769,"[0, 0, 1]"
1013770,lie_down_on_the_floor,lying_down,on_air,actor_3_walk_stick_full_ph_cam_7_5770,"[0, 0, 1]"


In [35]:
train_set["frame_name"] += ".jpg"
val_set["frame_name"] += ".jpg"

,micro_labels,macro_labels,ar_labels,frame_name,onehotlabels
206,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0206.jpg,"[1, 0, 0]"
207,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0207.jpg,"[1, 0, 0]"
208,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0208.jpg,"[1, 0, 0]"
209,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0209.jpg,"[1, 0, 0]"
210,sit_up_from_lying,adl,on_air,actor_1_bed_cam_1_0210.jpg,"[1, 0, 0]"
...,...,...,...,...,...
810180,lie_down_on_the_floor,lying_down,on_air,actor_2_walk_ph_cam_7_1260.jpg,"[0, 0, 1]"
810181,lie_down_on_the_floor,lying_down,on_air,actor_2_walk_ph_cam_7_1261.jpg,"[0, 0, 1]"
810182,lie_down_on_the_floor,lying_down,on_air,actor_2_walk_ph_cam_7_1262.jpg,"[0, 0, 1]"
810183,lie_down_on_the_floor,lying_down,on_air,actor_2_walk_ph_cam_7_1263.jpg,"[0, 0, 1]"


## Fine tunframe_nameG16 on Actor 4

### Instantiate Training and Validation Generators

In [43]:
# istantiate train ImageDataGenerator and define image augmentations.
train_generator = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# flow from dataframe loads frames from provided df, which is Actor 4 subset after balancing.
train_datagen = train_generator.flow_from_dataframe(
    train_set,
    directory=extracted_frames_path,
    x_col="frame_name",
    y_col="macro_labels",
    target_size=(224, 224),
    color_mode="rgb",
    classes=["adl", "lying_down", "falling"],
    class_mode="categorical",
    batch_size=64,
    shuffle=True,
    seed=2,
)

Found 70644 validated image filenames belonging to 3 classes.


In [44]:
# define validation ImageDataGenerator. No data augmentation is performed. The images are just rescaled to (0,1)
val_generator = ImageDataGenerator(rescale=1.0 / 255)

val_datagen = val_generator.flow_from_dataframe(
    val_set,
    directory=extracted_frames_path,
    x_col="frame_name",
    y_col="macro_labels",
    target_size=(224, 224),
    color_mode="rgb",
    classes=["adl", "lying_down", "falling"],
    class_mode="categorical",
    batch_size=64,
    shuffle=False,
    seed=2,
)

Found 79709 validated image filenames belonging to 3 classes.


In [45]:
# print samples per class identified from train generator
print(pd.Series(train_datagen.classes).value_counts())
print(train_datagen.class_indices)

0    23548
1    23548
2    23548
dtype: int64
{'adl': 0, 'falling': 1, 'lying_down': 2}


In [46]:
# print samples per class identified from val generator
print(pd.Series(val_datagen.classes).value_counts())
print(val_datagen.class_indices)

0    61502
1    14469
2     3738
dtype: int64
{'adl': 0, 'falling': 1, 'lying_down': 2}


### Download pre-trained VGG16

In [ ]:
# define image size 
IMG_SIZE = (224, 224, 3)

In [47]:
# Download VGG16 pre-trained on ImageNet
feature_extractor = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=IMG_SIZE,
    pooling="avg",
)

feature_extractor.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Freeze all layers except last convolutional block

In [48]:
# freeze all layers except last five
for layer in feature_extractor.layers[:-9]:
    layer.trainable = False

for i, layer in enumerate(feature_extractor.layers):
    print(i, layer.name, layer.trainable)

0 input_2 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 True
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 global_average_pooling2d_1 True


### Add top layer to fine tune

In [49]:
# adding two dense layers (512 and 256 units). Final dense layer with 3 neurons.
x = feature_extractor.output
x = Flatten()(x)
x = Dense(units=512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(units=256, activation="relu")(x)
x = Dense(units=3, activation="softmax")(x)

transfer_model = Model(inputs=feature_extractor.input, outputs=x)

transfer_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Compile the model and define callbacks

In [53]:
# compile the model and keep track of each class's recall
transfer_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=[
        "categorical_accuracy",
        tf.keras.metrics.Recall(class_id=0),
        tf.keras.metrics.Recall(class_id=1),
        tf.keras.metrics.Recall(class_id=2),
    ],
)

In [54]:
# checkpoint the model every epoch if val_loss improves
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f"{projectdir}/vision/model_checkpoints/VGG/VGG_finetune_actor_12.h5",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    save_freq="epoch",
)

In [57]:
callbacks = [
    model_checkpoint,
    tf.keras.callbacks.ReduceLROnPlateau(
        verbose=1
    ),  # reduces optimizer's lr if val_loss stops decreasing
    tf.keras.callbacks.EarlyStopping(
        patience=5, verbose=1
    ),  # early stops the training if val_loss increases for 5 epochs straight
]

### Train the model

In [58]:
# fit the model for 100 epochs
history = transfer_model.fit(
    train_datagen, validation_data=val_datagen, epochs=100, callbacks=callbacks
)

2022-08-29 09:11:42.470350: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-08-29 09:11:44.280109: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-08-29 09:11:48.125307: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1104/1104 [==============================] - 719s 644ms/step - loss: 0.7652 - categorical_accuracy: 0.6082 - recall_3: 0.7224 - recall_4: 0.1932 - recall_5: 0.5306 - val_loss: 3.4991 - val_categorical_accuracy: 0.1725 - val_recall_3: 0.1129 - val_recall_4: 0.2263 - val_recall_5: 0.6873

Epoch 00001: val_loss improved from inf to 3.49906, saving model to /home/jovyan/work/MED_Fall/vision/model_checkpoints/VGG/VGG_finetune_actor_12.h5
Epoch 2/100
1104/1104 [==============================] - 703s 637ms/step - loss: 0.5022 - categorical_accuracy: 0.7585 - recall_3: 0.9373 - recall_4: 0.5320 - recall_5: 0.7759 - val_loss: 4.3399 - val_categorical_accuracy: 0.1574 - val_recall_3: 0.0947 - val_recall_4: 0.2133 - val_recall_5: 0.8890

Epoch 00002: val_loss did not improve from 3.49906
Epoch 3/100
1104/1104 [==============================] - 701s 635ms/step - loss: 0.4215 - categorical_accuracy: 0.8011 - recall_3: 0.9535 - recall_4: 0.6256 - recall_5: 0.8104 - val_loss: 7.0734 - val_categorical

### Evaluate the model

In [ ]:
# load best checkpoint
best_model = tf.keras.models.load_model(
    f"{projectdir}/vision/model_checkpoints/resample/vgg_top_fine_tuned_best_epoch_undersample.h5"
)

In [ ]:
# evaluate the model on the val set
best_model.evaluate(val_datagen)

In [ ]:
# plot the accuracies
plt.figure()
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="upper left")
plt.grid()
plt.show()

In [ ]:
# plot the losses
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="upper left")
plt.grid()
plt.show()

In [ ]:
y_preds_logits = best_model.predict(val_datagen)

In [ ]:
y_preds = np.argmax(y_preds_logits, axis=1)

In [ ]:
# print classification report
print(
    classification_report(
        y_true=val_datagen.classes,
        y_pred=y_preds,
        target_names=list(val_datagen.class_indices.keys()),
    )
)

In [ ]:
pd.Series(train_datagen.classes).value_counts()

In [ ]:
#%cp ~/work/MED_Fall/vision/model_checkpoints/resample/vgg_top_fine_tuned_best_epoch_undersample.h5 ~/work/persistent/

In [ ]:
load_images()

## Save Features Extractor

In [ ]:
best_model.summary()

In [ ]:
# detach the previously added dense layers
feature_extractor = Sequential(best_model.layers[:-5])
feature_extractor.summary()

In [ ]:
# freeze again al layers
for layer in feature_extractor.layers:
    layer.trainable = False
for layer in feature_extractor.layers:
    print(layer.trainable)

In [ ]:
# save the feature extractor
feature_extractor.save(f"{projectdir}/vision/models/vgg_feature_extractor.h5")

In [ ]:
# predict a fake image to check that output size is the correct number of features.
pred_test = feature_extractor.predict(np.ones(shape=(1, 224, 224, 3)))
print(pred_test.shape)